In [1]:
import torch.nn as nn
import torch.nn.functional as F

class KSOM(nn.Module):
    def __init__(self, input_size, map_size, lr, sigma):
        super(KSOM, self).__init__()
        self.input_size = input_size
        self.map_size = map_size
        self.lr = lr
        self.sigma = sigma
        self.weights = nn.Parameter(torch.randn(map_size[0], map_size[1], input_size))

    def forward(self, input):
        # Calculate distances between input and weights
        input = input.unsqueeze(1).unsqueeze(1)
        distances = torch.sqrt(torch.sum((input - self.weights)**2, dim=-1))

        # Find the closest neuron (BMU)
        bmu_position = torch.argmin(distances.view(-1, self.map_size[0]*self.map_size[1]), dim=1)
        bmu_row = bmu_position // self.map_size[1]
        bmu_col = bmu_position % self.map_size[1]

        # Calculate the neighborhood function
        sigma_squared = self.sigma**2
        exponent = -((torch.arange(self.map_size[0]).unsqueeze(1) - bmu_row)**2 + 
                     (torch.arange(self.map_size[1]).unsqueeze(0) - bmu_col)**2) / sigma_squared
        neighborhood = torch.exp(exponent).unsqueeze(-1)

        # Update the weights
        delta = input - self.weights
        lr_multiplier = self.lr * neighborhood
        self.weights += lr_multiplier * delta

        return bmu_row, bmu_col


In [2]:
def train_ksom(model, data_loader, num_epochs):
    for epoch in range(num_epochs):
        for i, data in enumerate(data_loader):
            input = data.to(device)

            # Forward pass
            bmu_row, bmu_col = model(input)

            # Backward pass
            loss = torch.mean(torch.sqrt(torch.sum((input - model.weights[bmu_row, bmu_col])**2, dim=-1)))
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Update learning rate and sigma
        model.lr *= 0.9
        model.sigma *= 0.9


In [3]:
from torch.utils.data import DataLoader, Dataset

class InputDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

data = 
data_loader = DataLoader(InputDataset(data), batch_size=1, shuffle=True)


TypeError: object of type 'ellipsis' has no len()